In [ ]:
# Convolutional Neural Network
# 합성곱 신경망의 특징 : 파라미터 공유(동일한 필터로 여러 위치의 합성곱을 계산, 파라미터 적음), 이동 등변성(데이터 내에 객체가 존재하는 위치나 방향이 변경되더라도 상관 없이 동일한 출력값을 얻음)

In [18]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

import torchvision
from torchvision import transforms

# RNN data set : !pip install -U datasets

In [2]:
# Convolutional Layer
conv = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1)
input_data = torch.randn(1, 3, 4, 4) # batch, channel, height, width
output = conv(input_data)

print(output.shape)             # torch.size([1,64,4,4])

torch.Size([1, 64, 4, 4])


In [3]:
# Pooling Layer (입력 데이터의 공간적 크기를 줄이고, 중요한 특징만 남기기 위해 사용 , MaxPooling, Average Pooling)
print(input_data[0][0])
print('\n')

# 2D Max pooling : 2 by 2 window, stride : 2
maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
maxpool_output = maxpool(input_data)
print('\n')
print(maxpool_output[0][0])
print(maxpool_output.shape)     # torch.Size([1,3,2,2])

# 2D Average Pooling : 2 by 2 window, stride : 2
avgpool = nn.AvgPool2d(kernel_size=2, stride=2)
avgpool_output = avgpool(input_data)
print('\n')
print(avgpool_output[0][0])
print(avgpool_output.shape)     # torch.Size([1,3,2,2])

tensor([[ 1.6645,  0.8533, -0.7849, -0.7953],
        [-0.9416,  2.2746, -0.6730,  1.1657],
        [ 0.4158,  2.4443, -2.2567, -0.6434],
        [-0.1270,  0.9320, -1.6835, -0.3649]])




tensor([[ 2.2746,  1.1657],
        [ 2.4443, -0.3649]])
torch.Size([1, 3, 2, 2])


tensor([[ 0.9627, -0.2719],
        [ 0.9163, -1.2371]])
torch.Size([1, 3, 2, 2])


In [4]:
# Simple CNN Model.

class simple_CNN_model(nn.Module):
    def __init__(self, input_channel_size, output_channel_size, num_classes):
        super().__init__()

        #합성곱 레이어 2층
        self.conv1 = nn.Conv2d(input_channel_size, 32, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(32, output_channel_size, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        #Global Average Pooling
        self.gap = nn.AdaptiveAvgPool2d(1)      # output 크기 1 by 1 로 축소

        #Fully Connected Layer
        self.fc = nn.Linear(output_channel_size, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)

        x = self.gap(x)   # ㅎ(batch, output_channel_size, 1, 1)

        x = x.view(x.size(0), -1)   #평탄화(flatten)
        x = self.fc(x)

        return x

In [5]:
def plot_losses(train_losses, val_losses):
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='Training loss', color='blue')
    plt.plot(val_losses, label='Validation loss', color='red')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.show()

In [6]:
# LOAD MNIST DATA
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor(), download=True)

image, label = train_dataset[0]
print(image.size())   # torch.size(1,28,28)
print(label)          # 5

# Data Loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

# parameter setitng
input_channel_size = 1      #흑백
output_channel_size = 64    #은닉층 유닛 수
num_classes = 10
learning_rate = 1e-2

# initialization Model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = simple_CNN_model(input_channel_size, output_channel_size, num_classes).to(device)

# loss fun, optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

100%|██████████| 9.91M/9.91M [00:10<00:00, 909kB/s] 
100%|██████████| 28.9k/28.9k [00:00<00:00, 57.9kB/s]
100%|██████████| 1.65M/1.65M [00:06<00:00, 245kB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 7.26MB/s]


torch.Size([1, 28, 28])
5


In [7]:
# Loss 기록
train_losses = []
val_losses = []

# Goal
num_epochs = 25

# Laernig
for epoch in range(num_epochs):
    model.train()            # flag
    runnig_loss = 0.0        # epoch마다 평균 loss값 측정

    for images, labels in train_loader:
        images = images.to(device)      #(batch, 1, 28, 28)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        runnig_loss += loss.item()

    train_losses.append(runnig_loss / len(train_loader))

    # Verification

    model.eval()    #flag
    val_loss = 0.0
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)      # (batch,1, 28, 28)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()

    val_losses.append(val_loss / len(test_loader))

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_losses[-1]:.4f}, Val Loss: {val_losses[-1]:.4f}')

#테스트셋을 활용해 모델의 정확도 평가하기
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Test Accuracy: {100 * correct / total}%')

Epoch [1/25], Train Loss: 2.2954, Val Loss: 2.2860
Epoch [2/25], Train Loss: 2.2701, Val Loss: 2.2490
Epoch [3/25], Train Loss: 2.2077, Val Loss: 2.1506
Epoch [4/25], Train Loss: 2.0771, Val Loss: 2.0111
Epoch [5/25], Train Loss: 1.9673, Val Loss: 1.9184
Epoch [6/25], Train Loss: 1.8764, Val Loss: 1.8187
Epoch [7/25], Train Loss: 1.7659, Val Loss: 1.6977
Epoch [8/25], Train Loss: 1.6436, Val Loss: 1.5669
Epoch [9/25], Train Loss: 1.5214, Val Loss: 1.4462
Epoch [10/25], Train Loss: 1.4010, Val Loss: 1.3214
Epoch [11/25], Train Loss: 1.2848, Val Loss: 1.2080
Epoch [12/25], Train Loss: 1.1812, Val Loss: 1.1482
Epoch [13/25], Train Loss: 1.0928, Val Loss: 1.0901
Epoch [14/25], Train Loss: 1.0195, Val Loss: 1.0150
Epoch [15/25], Train Loss: 0.9568, Val Loss: 0.9293
Epoch [16/25], Train Loss: 0.9036, Val Loss: 0.9109
Epoch [17/25], Train Loss: 0.8531, Val Loss: 0.8479
Epoch [18/25], Train Loss: 0.8103, Val Loss: 0.9029
Epoch [19/25], Train Loss: 0.7722, Val Loss: 0.7808
Epoch [20/25], Train 

In [9]:
# Recurrent Neural Network.  (똑같은 FC층이 반복되는 구조, 연속되는 입력데이터 )
from datasets import load_dataset

#IMDB 데이터셋 로드
dataset = load_dataset('imdb')

#text 데이터만 불러오기
train_texts = [x['text'] for x in dataset['train']]
test_texts = [x['text'] for x in dataset['test']]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [15]:
# 단어 집합 생성 (train data)
def word_tokenizer(text):
  return text.lower().split()

all_words_list = []
for text in train_texts:
  all_words_list.extend(word_tokenizer(text))

vocab = sorted(list(set(all_words_list))) # 평탄화된 리스트로 set 생성

word2idx = {w: i+2 for i, w in enumerate(vocab)}    # 0: pad, 1:unk
word2idx['<PAD>'] = 0         # default token number
word2idx['<UNK>'] = 1         # default token number

idx2word = {i: w for w, i in word2idx.items()}
vocab_size = len(word2idx) # 변수 이름 수정: vocab_size

print(vocab_size)
print(word2idx)

Output hidden; open in https://colab.research.google.com to view.

In [16]:
# 단어 단위로 토크나이징
max_len = 100

def encode_text(text, max_len=max_len):
  tokens = word_tokenizer(text)
  indices = [word2idx.get(w, 1) for w in tokens[:max_len]]

  if len(indices) < max_len:
    indices += [0] * (max_len - len(indices))
    return torch.tensor(indicies, dtype=torch.long)

def encode_label(label):
  return torch.tensor(label, dtype=torch.flaot)

In [17]:
from torch.utils.data import Dataset

class IMDBCharDataset(Dataset):
  def __init__(self, split):
    self.data = dataset[split]
  def __len__(self):
    return len(self.data)
  def __getitem__(self, idx):
    text = self.data[idx]['text']
    label = self.data[idx]['label']

    return encode_text(text), encode_label(label)

In [19]:
import torch.nn as nn

class simple_RNN(nn.Module):
  def __init__(self, vocab_size, embed_dim, hidden_dim, num_classes=1):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
    self.rnn = nn.RNN(embed_dim, hidden_dim, batch_first=True)
    # self.rnn = nn.GRU(embed_dim, hidden_dim, batch_first=True)
    # self.rnn = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
    self.fc = nn.Linear(hidden_dim, num_classes)

  def forward(self, x):
    #x : (batch, seq_len)
    emb = self.embedding(x)     # batch, seq_len, embed_dim
    _, h_n = self.rnn(emb)      # h_n : (1, batch, hidden_dim)
    out = self.fc(h_n.squeeze(0))     # batch, num_classes
    return out.squeeze(1)         # batch

In [ ]:
# hyper parameter
embed_dim = 64    #문자 임베딩 차원
hidden_dim = 16   # 은닉층 차원
batch_size = 64
lr = 1e-3
epochs = 10

from torch.utils.data import DataLoader

# dataset 만들기
train_dataset = IMDBCharDataset('train')
test_dataset = IMDBCharDataset('test')

# dataloader 준비
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# 모델 불러오기
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = simple_RNN(vocab_size, embed_dim, hidden_dim).to(device)

# 손실함수 및 옵티마이져
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)